In [2]:
5+5

10

In [3]:
import polars as pl
import builtins
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [4]:
# import plotly.io as pio
# pio.renderers.default = "png" 

In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


In [6]:
path = r"C:\Users\Rudra\Desktop\bihar-viz\data\BL05.parquet"

# Load the Data

In [7]:
def load_level_05_data(path):
    # Load the data
    df = pl.read_parquet(path)
    
    # Correct datatype
    df = df.with_columns(
            (pl.col("OutOfHome_Consumption_Quantity").cast(pl.Float64, strict=False)),
            (pl.col("OutOfHome_Consumption_Value").cast(pl.Float64, strict=False))
        )
    
    # Select useful columns only
    df = df.select([
        'FSU_Serial_No','Sector','NSS_Region','District','Stratum',
        'Sub_stratum','Panel','Sub_sample','FOD_Sub_Region',
        'Sample_SU_No','Sample_Household_No','Questionnaire_No',
        'Item_Code','OutOfHome_Consumption_Quantity','OutOfHome_Consumption_Value',
        'Total_Consumption_Quantity','Total_Consumption_Value'
    ])
    
    return df

In [8]:
df = load_level_05_data(path)
fdf = df.clone()

In [9]:
fdf.head(2)

FSU_Serial_No,Sector,NSS_Region,District,Stratum,Sub_stratum,Panel,Sub_sample,FOD_Sub_Region,Sample_SU_No,Sample_Household_No,Questionnaire_No,Item_Code,OutOfHome_Consumption_Quantity,OutOfHome_Consumption_Value,Total_Consumption_Quantity,Total_Consumption_Value
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,bool,i64,f64,f64,f64,i64
27865,2,101,14,12,2,5,1,1010,1,15,false,141,null,null,1.0,84
27865,2,101,14,12,2,5,1,1010,1,15,false,143,null,null,1.5,180


In [10]:
def category_mapping():
    return {
        129: "cereals",
        139: "cereal substitute",
        159: "pulses & products",
        179: "salt & sugar",
        169: "milk & milk products",
        219: "vegetables",
        239: "fruits (fresh)",
        249: "fruits (dry)",
        199: "egg, fish & meat",
        189: "edible oil",
        269: "spices",
        279: "beverages",
        289: "served processed food",
        299: "packaged processed food"
    }

In [11]:
def cereal_mapping():
    return {
        61: "rice-free",
        62: "wheat/atta-free",
        70: "coarse grains-free",
        101: "rice – PDS",
        102: "rice – other sources",
        103: "chira",
        105: "muri",
        106: "other rice products (khoi/lawa, etc.)",
        107: "wheat/atta – PDS",
        108: "wheat/atta – other sources",
        110: "maida",
        111: "suji/rawa",
        112: "vermicelli (sewai)",
        114: "other wheat products",
        1: "coarse grains – PDS",
        2: "coarse grains – other sources",
        122: "other cereals & products",
        129: "cereals: sub-total"
    }

# Category Load and Clean

In [12]:
def category_dict_to_dataframe(category_mapping:dict, fdf=fdf):
    """
    1. Create the dataframe form the dict
    2. Join with the original / Filter dataset
    3. Create totally new dataset for further analysis
        a. Only valid data 
        b. Then category, Out of Home and Total columns selected
        c. Aggregate the columns
        d. Provide a clean and valid dataset for further analysis.
    """
     
    # Create the dataframe form dictionary
    map_df = pl.DataFrame({
        "Item_Code": builtins.list(category_mapping.keys()),
        "category_mapped": builtins.list(category_mapping.values()),
    })

    # Join the category dataframe & whole dataset
    fdf = fdf.join(map_df, on="Item_Code", how="left")
    
    # Create a totally new dataframe with desire columns
    cat_df = fdf.filter(
        pl.col("category_mapped").is_not_null()
        ) \
        [
        'OutOfHome_Consumption_Quantity',
        'OutOfHome_Consumption_Value',
        'Total_Consumption_Quantity',
        'Total_Consumption_Value',
        "category_mapped"
        ] \
        .group_by("category_mapped").agg(
            pl.col("OutOfHome_Consumption_Value").sum().alias("out_home_value"),
            pl.col("OutOfHome_Consumption_Quantity").sum().alias("out_home_qty"),
            pl.col("Total_Consumption_Quantity").sum().alias("total_qty"),
            pl.col("Total_Consumption_Value").sum().alias("total_value")
        ).with_columns(
            out_of_home_avg_pice = pl.when(pl.col("out_home_qty") > 0) 
                            .then(pl.col("out_home_value") / pl.col("out_home_qty"))
                            .otherwise(0),
            total_avg_pice = pl.when(pl.col("total_qty") > 0) 
                            .then(pl.col("total_value") / pl.col("total_qty"))
                            .otherwise(0),
        ) 
        
    return cat_df

In [13]:
cat_df = category_dict_to_dataframe(cereal_mapping())


In [14]:
cat_df

category_mapped,out_home_value,out_home_qty,total_qty,total_value,out_of_home_avg_pice,total_avg_pice
str,f64,f64,f64,i64,f64,f64
"""wheat/atta-free""",0.0,0.0,50071.0,0,0.0,0.0
"""muri""",0.0,0.0,7025.588,401067,0.0,57.08661
"""suji/rawa""",0.0,0.0,5294.788,235353,0.0,44.449938
"""coarse grains – other sources""",13112.0,607.5,2026.9,49226,21.583539,24.286349
"""other wheat products""",0.0,0.0,156.4,5415,0.0,34.622762
…,…,…,…,…,…,…
"""maida""",0.0,0.0,5662.765,224121,0.0,39.578015
"""other rice products (khoi/lawa…",0.0,0.0,284.43,11424,0.0,40.16454
"""cereals: sub-total""",4.140426e6,132868.3,888887.082,21450998,31.16188,24.132422


In [15]:
NUMERIC_COLS = [
    "out_home_value",
    "out_home_qty",
    "total_qty",
    "total_value",
    "out_of_home_avg_pice",
    "total_avg_pice",
]


In [ ]:
def pca_kmeans_category_clustering(
    cat_df,
    feature_cols,
    n_components=2,
    n_clusters=4,
    random_state=42
):
    """
    Apply PCA + KMeans on category-level data
    
    Parameters
    ----------
    cat_df : polars.DataFrame or pandas.DataFrame
    feature_cols : list[str]
    n_components : int
    n_clusters : int
    
    Returns
    -------
    result_df : pandas.DataFrame
    pca_model : PCA
    kmeans_model : KMeans
    """
    
    if hasattr(cat_df, "to_pandas"):
        pdf = cat_df.to_pandas()
    else:
        pdf = cat_df.copy()
    
    labels = pdf["category_mapped"]
    
    X = pdf[feature_cols].fillna(0)
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    pca = PCA(n_components=n_components, random_state=random_state)
    X_pca = pca.fit_transform(X_scaled)
    
    kmeans = KMeans(
        n_clusters=n_clusters,
        n_init=10,
        random_state=random_state
    )
    clusters = kmeans.fit_predict(X_pca)
    
    result_df = pd.DataFrame({
        "category_mapped": labels,
        "PC1": X_pca[:, 0],
        "PC2": X_pca[:, 1],
        "cluster": clusters
    })
    
    return result_df, pca, kmeans


In [18]:
FEATURE_COLS = [
    "out_home_value",
    "out_home_qty",
    "total_qty",
    "total_value",
    "out_of_home_avg_pice",
    "total_avg_pice",
]

pca_df, pca_model, kmeans_model = pca_kmeans_category_clustering(
    cat_df=cat_df,
    feature_cols=FEATURE_COLS,
    n_components=2,
    n_clusters=2
)


In [19]:
import plotly.express as px

In [28]:
fig = px.scatter(
    pca_df,
    x="PC1",
    y="PC2",
    color="cluster",
    hover_name="category_mapped",
    hover_data=["category_mapped"],
    title="Category Clustering after PCA",
    color_continuous_scale='Bluered_r'
)

fig.show()

In [21]:
cluster_summary = (
    pca_df
    .merge(cat_df.to_pandas(), on="category_mapped")
    .groupby("cluster")[FEATURE_COLS]
    .mean()
)

cluster_summary

,out_home_value,out_home_qty,total_qty,total_value,out_of_home_avg_pice,total_avg_pice
cluster,,,,,,
0,3.290615e+03,100.792308,22912.005692,1.379619e+05,8.517352,34.564138
1,2.746025e+06,88142.100000,493306.030000,1.370283e+07,31.671816,30.756732
